In [ ]:
import tensorflow as tf

In [2]:
class lid_model(object):
    def __init__(vocab_size=10000, embed_dim=300, num_layers=3, max_gradient_norm=10., batch_size=32, learning_rate=0.5, 
                 learning_rate_decay_factor=0.5, keep_prob=0.75, use_lstm=True, forward_only=False,dtype=tf.float32):
        self.emb_dim=embed_dim
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.learning_rate = tf.Variable(
            float(learning_rate), trainable=False, dtype=dtype)
        self.learning_rate_decay_op = self.learning_rate.assign(
            self.learning_rate * learning_rate_decay_factor)
        self.global_step = tf.Variable(0, trainable=False)
        self.cell=tf.contrib.rnn.BasicLSTMCell(dim)
    def dropout(self, inp):
        return tf.nn.dropout(inp, self.keep_prob)

    def add_place_holders(self):
        self.x=tf.placeholder(tf.int32, shape=[None, None],name="x")
        self.x_mask=tf.placeholder(tf.int32, shape=[None, None],name="x_mask")
        self.y=tf.placeholder(tf.int32, shape=[None],name="y")
    def add_word_embeddings_op(self):
        """Defines self.word_embeddings

        If self.config.embeddings is not None and is a np array initialized
        with pre-trained word vectors, the word embeddings is just a look-up
        and we don't train the vectors. Otherwise, a random matrix with
        the correct shape is initialized.
        """
        with tf.variable_scope("Embedding"):
            if self.config.embeddings is None:
                self.logger.info("WARNING: randomly initializing word vectors")
                _word_embeddings = tf.get_variable(
                        name="_word_embeddings",
                        dtype=tf.float32,
                        shape=[self.config.nwords, self.config.dim_word])
            else:
                _word_embeddings = tf.Variable(
                        self.config.embeddings,
                        name="_word_embeddings",
                        dtype=tf.float32,
                        trainable=self.config.train_embeddings)

            word_embeddings = tf.nn.embedding_lookup(_word_embeddings,
                    self.x, name="word_embeddings")
        with tf.variable_scope("RNN"):
            outputs=self.multilayer_bi_rnn(self.cell,word_embeddings,mask,scope.scope())
        '''with tf.variable_scope("chars"):
            if self.config.use_chars:
                # get char embeddings matrix
                _char_embeddings = tf.get_variable(
                        name="_char_embeddings",
                        dtype=tf.float32,
                        shape=[self.config.nchars, self.config.dim_char])
                char_embeddings = tf.nn.embedding_lookup(_char_embeddings,
                        self.char_ids, name="char_embeddings")

                # put the time dimension on axis=1
                s = tf.shape(char_embeddings)
                char_embeddings = tf.reshape(char_embeddings,
                        shape=[s[0]*s[1], s[-2], self.config.dim_char])
                word_lengths = tf.reshape(self.word_lengths, shape=[s[0]*s[1]])

                # bi lstm on chars
                cell_fw = tf.contrib.rnn.LSTMCell(self.config.hidden_size_char,
                        state_is_tuple=True)
                cell_bw = tf.contrib.rnn.LSTMCell(self.config.hidden_size_char,
                        state_is_tuple=True)
                _output = tf.nn.bidirectional_dynamic_rnn(
                        cell_fw, cell_bw, char_embeddings,
                        sequence_length=word_lengths, dtype=tf.float32)

                # read and concat output
                _, ((_, output_fw), (_, output_bw)) = _output
                output = tf.concat([output_fw, output_bw], axis=-1)

                # shape = (batch size, max sentence length, char hidden size)
                output = tf.reshape(output,
                        shape=[s[0], s[1], 2*self.config.hidden_size_char])
                word_embeddings = tf.concat([word_embeddings, output], axis=-1) '''

        self.word_embeddings =  self.dropout(word_embeddings)
    def bidirectional_rnn(self, cell, inputs, lengths, scope=None):
        name = scope.name or "BiRNN"
        # Forward direction
        with vs.variable_scope(name + "_FW") as fw_scope:
            output_fw, output_state_fw = rnn.dynamic_rnn(cell, inputs, time_major=False, dtype=dtypes.float32,
                                                       sequence_length=lengths, scope=fw_scope)
        # Backward direction
        inputs_bw = tf.reverse_sequence(inputs, tf.to_int64(lengths), seq_dim=0, batch_dim=1)
        with vs.variable_scope(name + "_BW") as bw_scope:
            output_bw, output_state_bw = rnn.dynamic_rnn(cell, inputs_bw, time_major=False, dtype=dtypes.float32,
                                                       sequence_length=lengths, scope=bw_scope)

        output_bw = tf.reverse_sequence(output_bw, tf.to_int64(lengths), seq_dim=0, batch_dim=1)

        outputs = output_fw + output_bw
        output_state = output_state_fw + output_state_bw

        return (outputs, output_state)
    def multilayer_bi_rnn(self, cell, inputs, mask, scope=None):
        inp = inputs
        for i in range(self.num_layers):
            with vs.variable_scope("Cell%d" % i) as scope:
                srclen = tf.reduce_sum(mask,axis=0)
                out, _ = self.bidirectional_rnn(cell, inp, srclen, scope=scope)
                out=tf.multiply(out,tf.expand_dims(mask,-1))
                inp = self.dropout(dropin)
        return out

SyntaxError: invalid syntax (<ipython-input-2-5e225b3df008>, line 98)